## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [16]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [17]:
# 데이터 로드
DI_train = pd.read_csv('../data/DI_train_dataset_35.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_35.csv')

In [18]:
# ID 열을 제외한 특성과 타겟 변수 분리
DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

In [19]:
print(f"DI_X shape: {DI_X.shape}")
print(f"DI_test shape: {DI_test.drop('ID', axis=1).shape}")

DI_X shape: (6290, 31)
DI_test shape: (2176, 31)


### 인코딩 

In [20]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "정자_기증자_나이"
]

In [21]:
# 모든 범주형 변수를 문자열로 변환
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

## Modeling

In [22]:
# 데이터 분할
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

### DI 데이터

In [23]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.00005, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 200.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 200.0),

        'random_state': 42,
        'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1   
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(DI_X_train, DI_y_train)
    
    y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
    auc = roc_auc_score(DI_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-17 14:49:00,974] A new study created in memory with name: no-name-642c9840-c505-4466-884d-dbe07db25b35
[I 2025-02-17 14:49:01,745] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 4081, 'num_leaves': 3268, 'learning_rate': 0.3541599356347277, 'min_child_samples': 252, 'reg_alpha': 199.12099857753972, 'reg_lambda': 181.607841923034}. Best is trial 0 with value: 0.5.
[I 2025-02-17 14:49:02,666] Trial 1 finished with value: 0.639176496286588 and parameters: {'n_estimators': 2773, 'num_leaves': 1061, 'learning_rate': 0.29139391171390727, 'min_child_samples': 180, 'reg_alpha': 100.71356361186521, 'reg_lambda': 121.0063987967541}. Best is trial 1 with value: 0.639176496286588.
[I 2025-02-17 14:49:03,484] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 2674, 'num_leaves': 2781, 'learning_rate': 0.4968242137424675, 'min_child_samples': 481, 'reg_alpha': 177.84284123804656, 'reg_lambda': 137.6816842768401}. Best is trial 1 with value: 0.639176496286

: 

In [ ]:
# import optuna
# import lightgbm as lgb
# from sklearn.metrics import roc_auc_score

# # 목적 함수 정의
# def objective(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 500, 9000),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
#         'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
#         'subsample': trial.suggest_float('subsample', 0.2, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),

#         'random_state': 42,
#         'scale_pos_weight': 6.757,
#         'objective': 'binary',
#         'metric': 'auc',
#         'verbose': -1     
#     }

#     model = lgb.LGBMClassifier(**param)
#     model.fit(DI_X_train, DI_y_train)
    
#     y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
#     auc = roc_auc_score(DI_y_test, y_pred_proba)
#     return auc

# # Optuna 스터디 생성 및 최적화 실행
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=2000)

# # 최적의 하이퍼파라미터 출력
# print("Best trial:")
# trial = study.best_trial
# print(f"  Value: {trial.value}")
# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

.